In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
from jax.scipy.interpolate import RegularGridInterpolator
sys.path.insert(0, '../../synthPy_restructured') 
import domain as d


In [2]:
# define some extent, the domain should be distributed as +extent to -extent, does not need to be cubic
extent_x = 5e-3
extent_y = 5e-3
extent_z = 10e-3

n_cells = 100

#x = np.linspace(-extent_x, extent_x, n_cells)
#y = np.linspace(-extent_y, extent_y, n_cells)
#z = np.linspace(-extent_z, extent_z, n_cells)

probing_extent = extent_z
probing_direction = 'z'

lengths = 2 * np.array([extent_x, extent_y, extent_z])

#domain = d.ScalarDomain(x = x, y = y, z = z, extent= probing_extent, probing_direction = probing_direction)     # create domain
# Much simpler domain function, no longer needlessly takes in beam values, they are fully seperated
domain = d.ScalarDomain(lengths, n_cells) 

In [3]:
def distance(x,y,z):
    return np.sqrt(x**2 + y**2 + z**2)

In [ ]:
def spherical(domain, r1, r2, r3, ne1, rho1, ne2, rho2, ne3, rho3):
    domain.spherical = True
    domain.densities = [0.0, rho1, rho2, rho3]
    ne = np.zeros_like(domain.XX)
    rho = np.zeros_like(domain.XX)
    for i in np.ndindex(domain.XX.shape):
        if distance(domain.XX[i], domain.YY[i], domain.ZZ[i]) <= r1:
            ne[i] = ne1
            rho[i] = rho1
        if r1 < distance(domain.XX[i], domain.YY[i], domain.ZZ[i]) <= r2:
            ne[i] = ne2
            rho[i] = rho2
        if r2 < distance(domain.XX[i], domain.YY[i], domain.ZZ[i]) <= r3:
            ne[i] = ne3
            rho[i] = rho3
        
    return ne,rho

In [ ]:
def spherical_2(domain, radii, ne, rho):
    radii = (0,) + radii
    domain.spherical = True
    domain.densities = np.array(rho)
    ne_grid = np.zeros_like(domain.XX)
    rho_grid = np.zeros_like(domain.XX)
    distance = distance(domain.XX, domain.YY, domain.ZZ)
    for i in range (0, len(radii)-1):
        ne_grid[radii[i] < distance <= radii[i+1]] = ne[i]
        rho_grid[radii[i] < distance <= radii[i+1]] = rho[i]
    
    return ne_grid, rho_grid

In [ ]:
def spherical_interps(Propagator, num_layers, files):
    opa_max=Propagator.ScalarDomain.x_n/(Propagator.ScalarDomain.x_length)
    Propagator.opacity_spatial_interps = []
    for i in range (num_layers):
        grp_centres, grps, rho, Te, opa_data = open_emi_files(f"../../{files[i]}")
        opa_data_capped = np.minimum(opa_max, opa_data)
        Propagator.opacity_interp = RegularGridInterpolator((grp_centres, rho, Te), opa_data_capped, bounds_error = False, fill_value = 0.0)
        Propagator.rho_interp = RegularGridInterpolator(
            (Propagator.ScalarDomain.x, Propagator.ScalarDomain.y, Propagator.ScalarDomain.z), Propagator.ScalarDomain.rho, bounds_error = False, fill_value = 0.0, method = "nearest")
        opacity_grid = Propagator.opacity_interp((Propagator.energy, Propagator.ScalarDomain.rho, Propagator.ScalarDomain.Te))
        Propagator.opacity_spatial_interps.append(RegularGridInterpolator((
            Propagator.ScalarDomain.x, Propagator.ScalarDomain.y, Propagator.ScalarDomain.z), opacity_grid, bounds_error = False, fill_value = 0.0))

In [ ]:
def spherical_atten(Propagator, x):
    density_values = Propagator.rho_interp(x.T)
    index = np.array([np.where(np.isclose(
    Propagator.ScalarDomain.densities, num)) for num in density_values])
    opacity = Propagator.opacity_spatial_interps[index](x.T)
    return -opacity*c
        